In [2]:
import cv2
import numpy as np

def estimate_motion_blur_kernel(size, angle, thickness=1):
    """
    生成運動模糊核
    """
    kernel = np.zeros((size, size))
    center = size // 2
    cv2.line(kernel, (center, 0), (center, size - 1), 1, thickness=thickness)
    rotation_matrix = cv2.getRotationMatrix2D((center, center), angle, 1)
    kernel = cv2.warpAffine(kernel, rotation_matrix, (size, size))
    kernel /= kernel.sum()
    return kernel

def wiener_filter(image, kernel, K):
    """
    使用維納濾波還原模糊影像
    """
    result = np.zeros_like(image)
    for i in range(image.shape[2]):  # 對每個通道處理
        channel = image[:, :, i]
        kernel_padded = np.pad(kernel, [(0, channel.shape[0] - kernel.shape[0]), 
                                        (0, channel.shape[1] - kernel.shape[1])], 
                               mode='constant')
        kernel_fft = np.fft.fft2(kernel_padded)
        channel_fft = np.fft.fft2(channel)
        kernel_conj = np.conj(kernel_fft)
        wiener_result = (kernel_conj / (np.abs(kernel_fft)**2 + K)) * channel_fft
        result[:, :, i] = np.abs(np.fft.ifft2(wiener_result))
    return result

def update(val):
    """
    更新還原結果
    """
    size = cv2.getTrackbarPos('Kernel Size', 'Motion Blur Restoration')
    angle = cv2.getTrackbarPos('Angle', 'Motion Blur Restoration')
    noise = cv2.getTrackbarPos('Noise (K)', 'Motion Blur Restoration') / 10000.0
    thickness = cv2.getTrackbarPos('Thickness', 'Motion Blur Restoration')

    # 防止 size 為偶數或太小
    size = max(3, size | 1)
    thickness = max(1, thickness)

    # 更新模糊核
    kernel = estimate_motion_blur_kernel(size, angle, thickness)

    # 更新還原影像
    restored = wiener_filter(blurred_img, kernel, noise)

    # 顯示更新結果
    restored_disp = cv2.convertScaleAbs(restored)
    cv2.imshow('Restored Image', restored_disp)

# 設定影像路徑
image_path = 'carplate.jpg'

# 讀取彩色影像
original_img = cv2.imread(image_path, cv2.IMREAD_COLOR)
if original_img is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

print(f"Image shape: {original_img.shape}")  # 檢查影像尺寸
blurred_img = original_img.copy()

# 創建 OpenCV 視窗
cv2.namedWindow('Motion Blur Restoration')
cv2.imshow('Original Image', blurred_img)

# 創建滑桿
cv2.createTrackbar('Kernel Size', 'Motion Blur Restoration', 15, 50, update)
cv2.createTrackbar('Angle', 'Motion Blur Restoration', 36, 360, update)
cv2.createTrackbar('Noise (K)', 'Motion Blur Restoration', 1, 10000, update)
cv2.createTrackbar('Thickness', 'Motion Blur Restoration', 1, 10, update)

# 初始化結果視窗
update(0)

# 等待用戶操作
cv2.waitKey(0)
cv2.destroyAllWindows()


FileNotFoundError: Image not found at path: carplate.jpg

In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import math

# 初始化全局變數
x_start, y_start, x_end, y_end = None, None, None, None

def calculate_angle(event):
    """
    畫布中的事件處理函數，用於計算線段的角度
    """
    global x_start, y_start, x_end, y_end
    if x_start is None and y_start is None:
        # 設定起點
        x_start, y_start = event.x, event.y
    else:
        # 設定終點
        x_end, y_end = event.x, event.y

        # 計算與水平線的角度
        dx = x_end - x_start
        dy = y_start - y_end  # 注意 Y 軸方向
        angle = math.degrees(math.atan2(dy, dx))  # 計算角度

        # 顯示結果
        angle_label.config(text=f"Angle: {angle:.2f}°")

        # 繪製線條
        canvas.create_line(x_start, y_start, x_end, y_end, fill="blue", width=2)

        # 重置起點
        x_start, y_start = None, None

def reset_canvas():
    """
    清除畫布並重置所有變數
    """
    global x_start, y_start, x_end, y_end
    x_start, y_start, x_end, y_end = None, None, None, None
    canvas.delete("all")
    canvas.create_image(0, 0, anchor=tk.NW, image=tk_image)
    angle_label.config(text="Angle: N/A")

# 創建 Tkinter 主視窗
root = tk.Tk()
root.title("Draw on Image and Calculate Angle")

# 讀取影像
image_path = 'carplate.jpg'
original_img = Image.open(image_path)

# 截取區域（模糊部分）
cropped_img = original_img.convert("L")  # x1, y1, x2, y2

# 轉換為 Tkinter 可用的影像
tk_image = ImageTk.PhotoImage(cropped_img)

# 創建畫布
canvas = tk.Canvas(root, width=cropped_img.width, height=cropped_img.height, bg="white")
canvas.pack()

# 在畫布上顯示影像
canvas.create_image(0, 0, anchor=tk.NW, image=tk_image)

# 綁定滑鼠事件
canvas.bind("<Button-1>", calculate_angle)

# 顯示角度的標籤
angle_label = tk.Label(root, text="Angle: N/A", font=("Arial", 14))
angle_label.pack()

# 清除按鈕
reset_button = tk.Button(root, text="Reset", command=reset_canvas, font=("Arial", 12))
reset_button.pack()

# 啟動 Tkinter 主循環
root.mainloop()


In [6]:
import cv2
# 讀取彩色影像
original_img = cv2.imread('C:/Users/evan6/OneDrive - 淡江大學/蕭兆翔博班/課程Course/2024影像處理/05_模糊/dai0.jpg')
print(original_img)

None
